# Imports and config

In [1]:
import os
import pickle
import random
import time
import itertools

import numpy as np
import pandas as pd

import xgboost as xgb

import config as cfg

# Model Selection

https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/  

We will use an approach similar to that of GBM here. The various steps to be performed are:

- Choose a relatively high learning rate. Generally a learning rate of 0.1 works but somewhere between 0.05 to 0.3 - should work for different problems. Determine the optimum number of trees for this learning rate. XGBoost has a very useful function called as “cv” which performs cross-validation at each boosting iteration and thus returns the optimum number of trees required.  
- Tune tree-specific parameters ( max_depth, min_child_weight, gamma, subsample, colsample_bytree) for decided learning rate and number of trees. Note that we can choose different parameters to define a tree and I’ll take up an example here.  
- Tune regularization parameters (lambda, alpha) for xgboost which can help reduce model complexity and enhance performance.  
- Lower the learning rate and decide the optimal parameters .

In [2]:
df= pd.read_pickle("cleaned_sets/df_fs_done.pkl")

In [3]:
train = df.loc[(df["date_block_num"]>=20) & (df["date_block_num"]<=32)]
test = df.loc[(df["date_block_num"]>=33)]

identificators = ["shop_id","item_id","date_block_num"]
label = "item_cnt_next_month"
predictors = [x for x in train.columns if x not in identificators and x!=label]

X_train = train[predictors]
y_train = train[label]

X_test = test[predictors+identificators]
del df

In [4]:
dtrain = xgb.DMatrix(X_train, label=y_train)

### Function for fine tuning

In [5]:
def dict_to_iterlist(d):
    keys=d.keys()
    lists=d.values()
    return list(keys), list(itertools.product(*lists))

In [6]:
def fine_tune_xgb(initial_params, gridsearch_params, dtrain, early_stopping_rounds=10, cv_fold=5):
    min_rmse = float("Inf")
    best_params = None
    
    params=initial_params
    gs_param_names, combinations = dict_to_iterlist(gridsearch_params)
    nb_gs_params = len(gs_param_names)
    
    boosting_rounds=100
    
    if "num_boost_round" in params:
        boosting_rounds = params["num_boost_round"]
    
    for combi in combinations:
        start_time=time.time()
        print(", ".join([f"{gs_param_names[i]}={combi[i]}" for i in range(nb_gs_params)]))

        # Update our parameters
        for i in range(nb_gs_params):
            params[gs_param_names[i]] = combi[i]
            
        if "num_boost_round" in gs_param_names:
            boosting_rounds=combi[gs_param_names.index("num_boost_round")]
        
        # Run CV
        cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=boosting_rounds,
            seed=42,
            nfold=cv_fold,
            metrics={'rmse'},
            early_stopping_rounds=early_stopping_rounds
        )
        # Update best RMSE
        
        mean_rmse = cv_results['test-rmse-mean'].min()
        boost_rounds = np.argmin(np.array(cv_results['test-rmse-mean']))
        
        print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
        print("Time taken for this round {}".format(time.time()-start_time))
        if mean_rmse < min_rmse:
            min_rmse = mean_rmse
            best_params = combi
    
    print(best_params)
    print("".join(["Best params:", 
                   ", ".join([f"{gs_param_names[i]}={best_params[i]}" for i in range(nb_gs_params)]),
                  f", RMSE: {min_rmse}",]))

#### Number of estimators

In [49]:
params = {
    # Parameters that we are going to tune.
    'max_depth':5,
    'min_child_weight': 2,
    'gamma':0,
    'eta':.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric':'rmse',
    "tree_method":'gpu_hist',
    "gpu_id":0
}

gridsearch_params = {
    "num_boost_round":[50,100,200,300,400,500]
        }

fine_tune_xgb(params, gridsearch_params, dtrain, early_stopping_rounds=20)

num_boost_round=50
[13:28:05] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[13:28:06] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[13:28:06] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[13:28:07] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[13:28:08] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
	RMSE 1.2196630000000002 for 49 rounds
Time taken for this round 103.70703482627869
num_boost_round=100
[13:29:48] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[13:29:49] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[13:29:50] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Par

num_boost_round=170 seems fine: more trees lead to the same results as the training is stopped at 167 rounds.

##### max_depth, min_child_weight

In [50]:
params = {
    # Parameters that we are going to tune.
    'num_boost_round':170,
    'max_depth':5,
    'min_child_weight': 2,
    'gamma':0,
    'eta':.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric':'rmse',
    "tree_method":'gpu_hist',
    "gpu_id":0
}

gridsearch_params = {
    "max_depth": [4,5,6,7,8,10,12],
    "min_child_weight": [1,2,3,4]
}

fine_tune_xgb(params, gridsearch_params, dtrain)

max_depth=4, min_child_weight=1
[13:56:21] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[13:56:22] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[13:56:23] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[13:56:23] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[13:56:24] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
	RMSE 1.2274471999999998 for 167 rounds
Time taken for this round 258.2041437625885
max_depth=4, min_child_weight=2
[14:00:39] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[14:00:39] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[14:00:40] WARNING: /home/robinblanchard/xgboos

[14:41:41] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[14:41:42] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[14:41:43] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[14:41:44] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[14:41:45] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
	RMSE 1.1802432 for 167 rounds
Time taken for this round 298.44205713272095
max_depth=7, min_child_weight=2
[14:46:40] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[14:46:41] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[14:46:41] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_

[15:28:44] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[15:28:45] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[15:28:46] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[15:28:48] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[15:28:49] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
	RMSE 1.1930223999999998 for 62 rounds
Time taken for this round 237.44577026367188
max_depth=12, min_child_weight=2
[15:32:41] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[15:32:43] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[15:32:44] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameter

In [52]:
params = {
    # Parameters that we are going to tune.
    'num_boost_round':170,
    'max_depth':5,
    'min_child_weight': 2,
    'gamma':0,
    'eta':.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric':'rmse',
    "tree_method":'gpu_hist',
    "gpu_id":0
}

gridsearch_params = {
    "max_depth": [7,8,9],
    "min_child_weight": [2,3,4]
}

fine_tune_xgb(params, gridsearch_params, dtrain)

max_depth=7, min_child_weight=2
[16:03:07] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[16:03:08] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[16:03:08] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[16:03:09] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[16:03:10] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
	RMSE 1.1804876 for 121 rounds
Time taken for this round 238.43140172958374
max_depth=7, min_child_weight=3
[16:07:05] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[16:07:07] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[16:07:07] WARNING: /home/robinblanchard/xgboost/src/le

max_depth=8, min_child_weight=2 are the best results

#### Recalibrate num_boost_round

In [51]:
params = {
    # Parameters that we are going to tune.
    'max_depth':8,
    'min_child_weight': 2,
    'gamma':0,
    'eta':.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric':'rmse',
    "tree_method":'gpu_hist',
    "gpu_id":0
}

gridsearch_params = {
    'num_boost_round':[300]
        }

fine_tune_xgb(params, gridsearch_params, dtrain, early_stopping_rounds=20)

num_boost_round=300
[15:46:05] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[15:46:06] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[15:46:07] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[15:46:08] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[15:46:09] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
	RMSE 1.1737327999999998 for 179 rounds
Time taken for this round 358.37774634361267
(300,)
Best params:num_boost_round=300, RMSE: 1.1737327999999998


We'll take 200

#### Gamma

In [53]:
params = {
    # Parameters that we are going to tune.
    'num_boost_round': 200,
    'max_depth':8,
    'min_child_weight': 2,
    'gamma':0,
    'eta':.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric':'rmse',
    "tree_method":'gpu_hist',
    "gpu_id":0
}

gridsearch_params = {
    'gamma':[0,0.1,0.2, 0.5, 1, 3, 5, 10]
        }

fine_tune_xgb(params, gridsearch_params, dtrain)

gamma=0
[16:35:47] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[16:35:48] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[16:35:49] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[16:35:50] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[16:35:51] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
	RMSE 1.1740466 for 142 rounds
Time taken for this round 283.63626527786255
gamma=0.1
[16:40:31] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[16:40:32] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[16:40:33] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } 

gamma=0.5 is the best

#### Tune subsample and colsample_bytree

In [7]:
params = {
    # Parameters that we are going to tune.
    'num_boost_round': 200,
    'max_depth':8,
    'min_child_weight': 2,
    'gamma':0,
    'eta':.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric':'rmse',
    "tree_method":'gpu_hist',
    "gpu_id":0
}

gridsearch_params = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
        }

fine_tune_xgb(params, gridsearch_params, dtrain)

subsample=0.6, colsample_bytree=0.6
[17:39:10] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[17:39:11] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[17:39:12] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[17:39:13] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[17:39:15] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
	RMSE 1.1875862000000001 for 87 rounds
Time taken for this round 184.87776160240173
subsample=0.6, colsample_bytree=0.7
[17:42:13] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[17:42:14] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[17:42:15] WARNING: /home/robinblanchar

[18:18:02] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[18:18:03] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[18:18:04] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[18:18:05] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[18:18:06] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
	RMSE 1.1733042 for 86 rounds
Time taken for this round 214.87110257148743
subsample=0.9, colsample_bytree=0.7
[18:21:35] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[18:21:36] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[18:21:37] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { n

In [9]:
params = {
    # Parameters that we are going to tune.
    'num_boost_round': 200,
    'max_depth':8,
    'min_child_weight': 2,
    'gamma':0,
    'eta':.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric':'rmse',
    "tree_method":'gpu_hist',
    "gpu_id":0
}

gridsearch_params = {
    'subsample':[0.75, 0.8, 0.85],
    'colsample_bytree':[0.55, 0.6, 0.65]
        }

fine_tune_xgb(params, gridsearch_params, dtrain)

subsample=0.75, colsample_bytree=0.55
[18:37:33] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[18:37:35] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[18:37:36] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[18:37:37] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[18:37:38] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
	RMSE 1.1852896000000002 for 88 rounds
Time taken for this round 208.34545588493347
subsample=0.75, colsample_bytree=0.6
[18:41:02] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[18:41:03] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[18:41:04] WARNING: /home/robinblanc

### Regularization parameters

In [7]:
params = {
    # Parameters that we are going to tune.
    'num_boost_round': 200,
    'max_depth':8,
    'min_child_weight': 2,
    'gamma':0,
    'eta':.1,
    'subsample': 0.8,
    'colsample_bytree': 0.55,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric':'rmse',
    "tree_method":'gpu_hist',
    "gpu_id":0
}

gridsearch_params = {
    'reg_alpha': [1e-5, 1e-2, 0.1, 1, 100]
        }

fine_tune_xgb(params, gridsearch_params, dtrain)

reg_alpha=1e-05
[07:42:39] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[07:42:43] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[07:42:44] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[07:42:45] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[07:42:46] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
	RMSE 1.1705998 for 86 rounds
Time taken for this round 217.98684453964233
reg_alpha=0.01
[07:46:13] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[07:46:14] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[07:46:15] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_bo

In [10]:
params = {
    # Parameters that we are going to tune.
    'num_boost_round': 200,
    'max_depth':8,
    'min_child_weight': 2,
    'gamma':0,
    'eta':.1,
    'subsample': 0.8,
    'colsample_bytree': 0.55,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric':'rmse',
    "tree_method":'gpu_hist',
    "gpu_id":0
}

gridsearch_params = {
    'reg_lambda': [1e-5, 1e-2, 0.1, 1, 100]
        }

fine_tune_xgb(params, gridsearch_params, dtrain)

reg_lambda=1e-05
[08:15:25] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[08:15:26] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[08:15:27] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[08:15:28] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[08:15:29] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
	RMSE 1.1877206 for 79 rounds
Time taken for this round 200.7032380104065
reg_lambda=0.01
[08:18:45] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[08:18:46] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[08:18:47] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_b

In [8]:
params = {
    # Parameters that we are going to tune.
    'num_boost_round': 200,
    'max_depth':8,
    'min_child_weight': 2,
    'gamma':0,
    'eta':.1,
    'subsample': 0.8,
    'colsample_bytree': 0.55,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric':'rmse',
    "tree_method":'gpu_hist',
    "gpu_id":0
}

gridsearch_params = {
    'reg_lambda': [0,0.1, 0.5, 1, 2],
    'reg_alpha' : [0,0.1, 0.5, 1, 2]
        }

fine_tune_xgb(params, gridsearch_params, dtrain)

reg_lambda=0, reg_alpha=0
[10:09:34] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[10:09:35] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[10:09:36] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[10:09:37] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[10:09:38] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
	RMSE 1.1847988 for 84 rounds
Time taken for this round 209.03050017356873
reg_lambda=0, reg_alpha=0.1
[10:13:03] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[10:13:04] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[10:13:05] WARNING: /home/robinblanchard/xgboost/src/learner.cc:29

[10:50:31] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[10:50:32] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[10:50:33] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[10:50:34] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
	RMSE 1.1789546000000002 for 77 rounds
Time taken for this round 193.2494192123413
reg_lambda=0.5, reg_alpha=1
[10:53:43] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[10:53:44] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[10:53:45] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[10:53:47] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { n

[11:33:10] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[11:33:11] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[11:33:12] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
	RMSE 1.1820598 for 88 rounds
Time taken for this round 210.4863612651825
(1, 2)
Best params:reg_lambda=1, reg_alpha=2, RMSE: 1.1634102000000002


In [ ]:
params = {
    # Parameters that we are going to tune.
    'num_boost_round': 200,
    'max_depth':8,
    'min_child_weight': 2,
    'gamma':0,
    'eta':.1,
    'subsample': 0.8,
    'colsample_bytree': 0.55,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric':'rmse',
    "tree_method":'gpu_hist',
    "gpu_id":0
}

gridsearch_params = {
    'reg_lambda': [1, 2, 3],
    'reg_alpha' : [2, 3, 4]
        }

fine_tune_xgb(params, gridsearch_params, dtrain)

reg_lambda=1, reg_alpha=2
[11:38:56] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[11:38:57] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[11:38:58] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[11:38:59] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[11:39:00] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
	RMSE 1.1672677999999999 for 86 rounds
Time taken for this round 210.08526277542114
reg_lambda=1, reg_alpha=3
[11:42:26] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[11:42:27] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[11:42:28] WARNING: /home/robinblanchard/xgboost/src/learne

#### Reducing learning rate and increasing boosting rounds

In [7]:
params = {
    # Parameters that we are going to tune.
    'num_boost_round': 2000,
    'max_depth':8,
    'min_child_weight': 2,
    'gamma':0,
    'eta':.1,
    'subsample': 0.8,
    'colsample_bytree': 0.7,
    'reg_lambda':1,
    'reg_alpha':2,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric':'rmse',
    "tree_method":'gpu_hist',
    "gpu_id":0
}

gridsearch_params = {
        }

fine_tune_xgb(params, gridsearch_params, dtrain, early_stopping_rounds=20)


[13:47:41] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[13:47:42] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[13:47:43] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[13:47:44] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
[13:47:45] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { num_boost_round } are not used.
	RMSE 1.1810918000000001 for 75 rounds
Time taken for this round 203.28630375862122
()
Best params:, RMSE: 1.1810918000000001


## Training

In [26]:
params = {
    # Parameters that we are going to tune.
    'num_boost_round': 150,
    'max_depth':8,
    'min_child_weight': 2,
    'gamma':0,
    'eta':.1,
    'subsample': 0.8,
    'colsample_bytree': 0.7,
    'reg_lambda':1,
    'reg_alpha':2,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric':'rmse',
    "tree_method":'gpu_hist',
    "gpu_id":0
}

xgbr = xgb.XGBRegressor(max_depth = params["max_depth"], min_child_weight = params["min_child_weight"],
                       gamma = params["gamma"], eta=params["eta"], subsample=params["subsample"],
                       colsample_bytree= params["colsample_bytree"], n_estimators=params["num_boost_round"],
                       objective=params["objective"], tree_method= params["tree_method"], gpu_id = params["gpu_id"])

print(dtrain.feature_names)
xgbr.fit(X_train, y_train, eval_metric="rmse",early_stopping_rounds=20, eval_set=[(X_train, y_train)],verbose=True)

['days_with_sell', 'city_name_Moscow', 'city_name_Khimki', 'item_cnt_month', 'std_item_price_lag2', 'mean_item_price', 'mean_category_item_price', 'mean_city_cnt_month', 'std_category_item_price_lag1', 'nb_days', 'main_category_name_Others', 'std_item_price_lag1', 'city_name_SPb', 'city_name_Other', 'std_city_item_price', 'month', 'item_cnt_month_lag2', 'mean_city_item_price']
[13:56:25] WARNING: /home/robinblanchard/xgboost/src/learner.cc:296: Parameters: { base_score, importance_type, missing, n_estimators } are not used.
[0]	validation_0-rmse:2.25022
Will train until validation_0-rmse hasn't improved in 20 rounds.
[1]	validation_0-rmse:2.12050
[2]	validation_0-rmse:1.98549
[3]	validation_0-rmse:1.86912
[4]	validation_0-rmse:1.75886
[5]	validation_0-rmse:1.66426
[6]	validation_0-rmse:1.58306
[7]	validation_0-rmse:1.49978
[8]	validation_0-rmse:1.43010
[9]	validation_0-rmse:1.36563
[10]	validation_0-rmse:1.31378
[11]	validation_0-rmse:1.26338
[12]	validation_0-rmse:1.21995
[13]	validat

[241]	validation_0-rmse:0.72810
[242]	validation_0-rmse:0.72775
[243]	validation_0-rmse:0.72764
[244]	validation_0-rmse:0.72755
[245]	validation_0-rmse:0.72723
[246]	validation_0-rmse:0.72686
[247]	validation_0-rmse:0.72662
[248]	validation_0-rmse:0.72628
[249]	validation_0-rmse:0.72613
[250]	validation_0-rmse:0.72598
[251]	validation_0-rmse:0.72575
[252]	validation_0-rmse:0.72564
[253]	validation_0-rmse:0.72540
[254]	validation_0-rmse:0.72535
[255]	validation_0-rmse:0.72524
[256]	validation_0-rmse:0.72515
[257]	validation_0-rmse:0.72489
[258]	validation_0-rmse:0.72464
[259]	validation_0-rmse:0.72420
[260]	validation_0-rmse:0.72396
[261]	validation_0-rmse:0.72391
[262]	validation_0-rmse:0.72369
[263]	validation_0-rmse:0.72356
[264]	validation_0-rmse:0.72341
[265]	validation_0-rmse:0.72319
[266]	validation_0-rmse:0.72285
[267]	validation_0-rmse:0.72253
[268]	validation_0-rmse:0.72190
[269]	validation_0-rmse:0.72182
[270]	validation_0-rmse:0.72177
[271]	validation_0-rmse:0.72160
[272]	va

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, eta=0.1, gamma=0,
             gpu_id=0, importance_type='gain', learning_rate=0.1,
             max_delta_step=0, max_depth=8, min_child_weight=2, missing=None,
             n_estimators=300, n_jobs=1, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='gpu_hist', verbosity=1)

# Prediction

In [27]:
X_test = test[["shop_id","item_id","date_block_num"]+predictors]
print(X_test.isnull().sum())
X_test.head(2)

shop_id                         0
item_id                         0
date_block_num                  0
days_with_sell                  0
city_name_Moscow                0
city_name_Khimki                0
item_cnt_month                  0
std_item_price_lag2             0
mean_item_price                 0
mean_category_item_price        0
mean_city_cnt_month             0
std_category_item_price_lag1    0
nb_days                         0
main_category_name_Others       0
std_item_price_lag1             0
city_name_SPb                   0
city_name_Other                 0
std_city_item_price             0
month                           0
item_cnt_month_lag2             0
mean_city_item_price            0
dtype: int64


,shop_id,item_id,date_block_num,days_with_sell,city_name_Moscow,city_name_Khimki,item_cnt_month,std_item_price_lag2,mean_item_price,mean_category_item_price,...,std_category_item_price_lag1,nb_days,main_category_name_Others,std_item_price_lag1,city_name_SPb,city_name_Other,std_city_item_price,month,item_cnt_month_lag2,mean_city_item_price
33,0,16385,33,0.0,0,0,0.0,0.0,310.568627,1150.581824,...,12.951833,31,0,0.0,0,0,2.928609,10,0.0,668.887421
67,0,8195,33,0.0,0,0,0.0,0.0,141.206851,316.328258,...,0.777997,31,0,0.0,0,0,2.928609,10,0.0,668.887421


In [28]:
y_pred = xgbr.predict(X_test[predictors])
X_test["item_cnt_month_pred"] = y_pred

X_test.head(2)

,shop_id,item_id,date_block_num,days_with_sell,city_name_Moscow,city_name_Khimki,item_cnt_month,std_item_price_lag2,mean_item_price,mean_category_item_price,...,nb_days,main_category_name_Others,std_item_price_lag1,city_name_SPb,city_name_Other,std_city_item_price,month,item_cnt_month_lag2,mean_city_item_price,item_cnt_month_pred
33,0,16385,33,0.0,0,0,0.0,0.0,310.568627,1150.581824,...,31,0,0.0,0,0,2.928609,10,0.0,668.887421,0.041348
67,0,8195,33,0.0,0,0,0.0,0.0,141.206851,316.328258,...,31,0,0.0,0,0,2.928609,10,0.0,668.887421,0.016083


# Format to submission

In [29]:
to_pred = pd.read_csv(cfg.FILENAMES['TEST_SALES'])
to_pred.head(2)

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320


In [30]:
submission = pd.merge(to_pred, X_test[["item_id","shop_id","item_cnt_month_pred"]],
                      how="left", on=["item_id","shop_id"])
print(submission.isnull().sum())
submission = submission.fillna(0)
submission.head(2)

ID                          0
shop_id                     0
item_id                     0
item_cnt_month_pred    102796
dtype: int64


,ID,shop_id,item_id,item_cnt_month_pred
0,0,5,5037,0.988628
1,1,5,5320,0.000000


In [31]:
submission.loc[submission["item_cnt_month_pred"]>20,"item_cnt_month_pred"]=20
submission.loc[submission["item_cnt_month_pred"]<0, "item_cnt_month_pred"]=0

In [32]:
sub_example = pd.read_csv(cfg.FILENAMES["SAMPLE_SUBM"])
sub_example.head(2)

,ID,item_cnt_month
0,0,0.5
1,1,0.5


In [33]:
submission_formated = (submission[["ID", "item_cnt_month_pred"]]
                       .rename({"item_cnt_month_pred":"item_cnt_month"}, axis=1))
submission_formated.head(2)

,ID,item_cnt_month
0,0,0.988628
1,1,0.000000


In [34]:
submission_formated.to_csv(os.path.join("submissions", "sub_09.csv"), index=False)